# Tutorial of MMDetection (Version 3.3.0)

Please get in touch if interested in our future collaboration of science and/or business!\
Daiphys Technologies LLC - https://www.daiphys.com/

## 0. References

- https://github.com/open-mmlab/mmdetection/blob/master/docs/en/get_started.md/
- https://mmdetection.readthedocs.io/en/latest/get_started.html
- https://mmdetection.readthedocs.io/en/latest/1_exist_data_model.html
- https://dev.classmethod.jp/articles/mmdetection-detect-samples/
- https://github.com/open-mmlab/mmcv/issues/3059
- https://github.com/open-mmlab/mmdetection/issues/10380

## 1. Preparation

### 1.1. Install

In [ ]:
## Python
!python --version

In [ ]:
## PyTorch
!python -c 'import torch; print(torch.__version__)'

In [ ]:
## Downgrading PyTorch on Google Colab (2.3.0+cu121) for MMCV Compatibility (2024/07/04)
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
## PyTorch
!python -c 'import torch; print(torch.__version__)'

In [ ]:
## CUDA
!nvcc --version

In [ ]:
!nvidia-smi

In [ ]:
!apt-get update

In [ ]:
## Resolving the dependency conflicts for openmim: ipython 7.9.0 requires jedi>=0.10
!pip install --upgrade jedi

In [ ]:
!pip install --upgrade openmim
## Restarting the runtime after installing openmin

In [ ]:
!mim install mmengine

In [ ]:
#!mim install mmcv-full
#!mim install mmcv
!mim install 'mmcv==2.1.0'

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git

In [ ]:
!pip install -v -e mmdetection
#!pip install --upgrade mmdet

In [ ]:
!python -c 'import mmdet; print(mmdet.__version__)'

### 1.2. Import

In [ ]:
#!pip list
#!pip freeze > requirements.txt

In [ ]:
import os
import sys
import numpy as np
import scipy as sp
import pandas as pd
import cv2
import matplotlib
import matplotlib.pyplot as plt
import pickle
import re
import csv
import math
import time
import datetime

In [ ]:
#from google.colab.patches import cv2_imshow

In [ ]:
import base64
import IPython

In [ ]:
from PIL import Image

In [ ]:
import torch
import torchvision

In [ ]:
sys.path.append('mmdetection')

In [ ]:
import mmdet

In [ ]:
from mmdet.apis import init_detector
from mmdet.apis import inference_detector
#from mmdet.apis import show_result_pyplot

In [ ]:
from mmdet.utils import register_all_modules

In [ ]:
register_all_modules()

In [ ]:
from mmengine.visualization import Visualizer
from mmdet.registry import VISUALIZERS

In [ ]:
import mmcv

In [ ]:
#from mmcv.transforms import Compose

In [ ]:
%matplotlib inline
#plt.rcParams['figure.figsize'] = [12, 8]

### 1.3. Setup

In [ ]:
project = 'tutorial'

In [ ]:
config_dir = project + '/configs'
output_dir = project + '/outputs'
checkpoint_dir = project + '/checkpoints'

In [ ]:
if torch.cuda.is_available() :
    #device = 'cuda'
    device = 'cuda:0'
else :
    device = 'cpu'

In [ ]:
device

In [ ]:
img = 'mmdetection/demo/demo.jpg'
video = 'mmdetection/demo/demo.mp4'

## 2. Sample Data with Faster R-CNN

### 2.1. Image

In [ ]:
## https://github.com/open-mmlab/mmdetection/tree/main/configs/faster_rcnn
#!mim download mmdet --config faster_rcnn_r50_fpn_1x_coco --dest $config_dir
!mim download mmdet --config faster-rcnn_r50_fpn_1x_coco --dest $config_dir

In [ ]:
#config_file = config_dir + '/faster_rcnn_r50_fpn_1x_coco.py'
config_file = config_dir + '/faster-rcnn_r50_fpn_1x_coco.py'
checkpoint_file = config_dir + '/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

In [ ]:
model = init_detector(config_file, checkpoint_file, device=device)

In [ ]:
#model.dataset_meta

In [ ]:
%%time
result = inference_detector(model, img)

In [ ]:
result.pred_instances.scores
#result.pred_instances.labels
#result.pred_instances.bboxes

In [ ]:
## model.show_result was removed after MMDetection version 2 (2024/07/04)
#model.show_result(img, result)
#model.show_result(img, result, out_file=output_dir + '/faster-rcnn_r50_fpn_1x_coco_result.jpg')

In [ ]:
## show_result_pyplot was removed after MMDetection version 2 (2024/07/04)
#show_result_pyplot(model, img, result, score_thr=0.50)

In [ ]:
#image = mmcv.imread(img)
image = mmcv.imread(img, channel_order='rgb')
#image = mmcv.imconvert(image, 'bgr', 'rgb')

In [ ]:
## Visualization Option A for MMDetection version 3
visualizer = VISUALIZERS.build(model.cfg.visualizer)
visualizer.dataset_meta = model.dataset_meta
visualizer.add_datasample('result', image, data_sample=result, draw_gt=False, show=True, wait_time=0, pred_score_thr=0.70, out_file=output_dir + '/faster-rcnn_r50_fpn_1x_coco_result_01.jpg')

In [ ]:
visualizer.show()
#visualizer.get_image()

In [ ]:
## Visualization Option B for MMDetection version 3
pred_score_thr = 0.70
scores = result.pred_instances.scores[torch.where(result.pred_instances.scores>=pred_score_thr)]
labels = result.pred_instances.labels[torch.where(result.pred_instances.scores>=pred_score_thr)]
bboxes = result.pred_instances.bboxes[torch.where(result.pred_instances.scores>=pred_score_thr)]
labels_classes = list(map(lambda x: model.dataset_meta['classes'][x], labels.tolist()))
labels_palette = list(map(lambda x: model.dataset_meta['palette'][x], labels.tolist()))
labels_print = list(map(lambda x, y: f'{x}: {(y * 100.0):.1f}', labels_classes, scores.tolist()))
bboxes_origins = list(map(lambda x: [x[0], x[1]], bboxes.tolist()))
visualizer = Visualizer(image=image)
visualizer.draw_bboxes(bboxes, edge_colors=labels_palette, line_widths=3)
visualizer.draw_texts(labels_print, torch.tensor(bboxes_origins), font_sizes=8, colors='white', bboxes=dict(facecolor='red', edgecolor='black', linewidth=0, alpha=0.8))

In [ ]:
visualizer.show()
#visualizer.get_image()

In [ ]:
cv2.imwrite(output_dir + '/faster-rcnn_r50_fpn_1x_coco_result_02.jpg', cv2.cvtColor(visualizer.get_image(), cv2.COLOR_RGB2BGR))

### 2.2. Video

In [ ]:
#model.cfg.test_dataloader.dataset.pipeline[0].type = 'LoadImageFromNDArray'
#test_pipeline = Compose(model.cfg.test_dataloader.dataset.pipeline)

In [ ]:
visualizer = VISUALIZERS.build(model.cfg.visualizer)
visualizer.dataset_meta = model.dataset_meta

In [ ]:
video_reader = mmcv.VideoReader(video)

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

In [ ]:
video_writer = cv2.VideoWriter(output_dir + '/faster_rcnn_r50_fpn_1x_coco_result.mp4', fourcc, video_reader.fps, (video_reader.width, video_reader.height))

In [ ]:
for frame in video_reader :
    #frame = cv2.rotate(frame, cv2.ROTATE_180)
    result = inference_detector(model, frame)
    #frame = model.show_result(frame, result, score_thr=0.90)
    visualizer.add_datasample('video', frame, data_sample=result, draw_gt=False, show=False, wait_time=0, pred_score_thr=0.90)
    frame = visualizer.get_image()
    video_writer.write(frame)

In [ ]:
video_writer.release()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
mp4_data = open(output_dir + '/faster_rcnn_r50_fpn_1x_coco_result.mp4', 'rb').read()

In [ ]:
mp4_url = 'data:video/mp4;base64,' + base64.b64encode(mp4_data).decode()
mp4_html = '<video width=400 controls><source src="{mp4_url}" type="video/mp4"></video>'

In [ ]:
IPython.display.HTML(mp4_html)